In [1]:
using JuMP, Clp, Plots

In [2]:
plantacao = ["Wheat", "Corn", "Sugar Cane"]
produtos = ["Wheat", "Corn", "Sugar Cane High", "Sugar Cane Low"]
cenarios = [1, 2, 3]
preco_venda = [170, 150, 36, 10]
preco_producao = [150, 230, 260]
producao = [ 3    2.5  2  ;
             3.6  3    2.4;
            24   20   16]
preco_compra = [238, 210]
qtd_exigida = [200, 240]
terra_disponivel = 500
max_sugar_cane_high = 6000;

In [3]:
function Q(xW,xC,xS,ξ)
    m = Model(Clp.Optimizer)
    set_optimizer_attribute(m, "LogLevel",0)
    @variables(m, 
    begin
        zw
        zc
        zs
        W[produtos] >= 0
        Y[produtos[1:2]] >= 0
    end)

    @constraints(m,
    begin
        HERD_REQUIREMENTS_WHEAT, producao[1, ξ] * zw + Y["Wheat"] - W["Wheat"] >= qtd_exigida[1] 
        HERD_REQUIREMENTS_CORN, producao[2, ξ] * zc + Y["Corn"] - W["Corn"] >= qtd_exigida[2]
        SUGAR_CANE_IMPOSITIONS_1, sum(W[pr] for pr in produtos[3:4]) <= producao[3,ξ] * zs
        SUGAR_CANE_IMPOSITIONS_2, W[produtos[3]] <= max_sugar_cane_high
        auxW, zw == xW
        auxC, zc == xC
        auxS, zs == xS
    end)

    @objective(m, Min,sum(preco_compra[ipr] * Y[pr] for (ipr, pr) in enumerate(produtos[1:2]))
                     - sum(preco_venda[ipr] * W[pr] for (ipr, pr) in enumerate(produtos)))
    optimize!(m)
    return objective_value(m), shadow_price(auxW), shadow_price(auxC), shadow_price(auxS) 
end

Q (generic function with 1 method)

In [4]:
master = Model(Clp.Optimizer)
set_optimizer_attribute(master, "LogLevel",0)
@variable(master,x[plantacao] >= 0)
@variable(master,α)
@constraint(master, α >= -999999)
@constraint(master,LAND_AVAILABLE, sum(x[pl] for pl in plantacao) <= terra_disponivel)
@objective(master, Min, sum(preco_producao[ipl] * x[pl] for (ipl, pl) in enumerate(plantacao)) + α)
optimize!(master);

iterator = 1
Q̂i_list = []
πi_list = []
x̂i_list = []
LB_list = []
UB_list = []
LB = nothing
UB = nothing
x̂i = nothing

while true
    LB = objective_value(master)
    if iterator == 1
        x = master[:x]
        x̂i = value.(x)
        append!(x̂i_list,[x̂i["Wheat"],x̂i["Corn"],x̂i["Sugar Cane"]])
    else
        aux = copy(master)
        set_optimizer(aux, Clp.Optimizer)
        set_optimizer_attribute(aux, "LogLevel",0)
        x_aux = aux[:x]
        γ = 0.5
        obj = objective_function(aux)
        @variable(aux, θ)
        @constraints(aux,begin
                obj <= γ*LB + (1-γ)*UB
                θ >= x_aux["Wheat"]-x̂i["Wheat"]
                θ >= -(x_aux["Wheat"]-x̂i["Wheat"])
                θ >= x_aux["Corn"]-x̂i["Corn"]
                θ >= -(x_aux["Corn"]-x̂i["Corn"])
                θ >= x_aux["Sugar Cane"]-x̂i["Sugar Cane"]
                θ >= -(x_aux["Sugar Cane"]-x̂i["Sugar Cane"])
            end)
        @objective(aux, Min, θ)
        optimize!(aux)
        x̂i = value.(x_aux)
        append!(x̂i_list,[x̂i["Wheat"],x̂i["Corn"],x̂i["Sugar Cane"]])
    end

    slave_Q̄i_list = []
    slave_πiW_list = []
    slave_πiC_list = []
    slave_πiS_list = []
    for ξ in cenarios
        q̂, pîW,pîC,pîS = Q(x̂i["Wheat"],x̂i["Corn"],x̂i["Sugar Cane"],ξ)
        append!(slave_Q̄i_list, q̂)
        append!(slave_πiW_list, pîW)
        append!(slave_πiC_list, pîC)
        append!(slave_πiS_list, pîS)
    end

    Q̂i = sum(slave_Q̄i_list)/size(slave_Q̄i_list,1)
    πiW = sum(slave_πiW_list)/size(slave_πiW_list,1)
    πiC = sum(slave_πiC_list)/size(slave_πiC_list,1)
    πiS = sum(slave_πiS_list)/size(slave_πiS_list,1)

    l_i = (x1,x2,x3) -> Q̂i + πiW*(x1-x̂i["Wheat"]) + πiC*(x2-x̂i["Corn"]) + πiS*(x3-x̂i["Sugar Cane"])

    append!(Q̂i_list, Q̂i)
    append!(πi_list, [πiW,πiC,πiS])

    UB = sum(preco_producao[ipl] * x̂i[pl] for (ipl, pl) in enumerate(plantacao)) + Q̂i
    append!(LB_list, LB)
    append!(UB_list, UB)

    if UB-LB <0.01
        println("**SOLUTION FOUND**")
        println( """Iteration $(iterator): x̂$(iterator)["Wheat"]: $(x̂i["Wheat"]) & x̂$(iterator)["Corn"]: $(x̂i["Corn"]) & x̂$(iterator)["SugarCane"]: $(x̂i["Sugar Cane"])""")
        println( "Lower Bound: $(LB) & Uper Bound: $(UB)")
        println("===========")
        break
    else
        println( """Iteration $(iterator): x̂$(iterator)["Wheat"]: $(x̂i["Wheat"]) & x̂$(iterator)["Corn"]: $(x̂i["Corn"]) & x̂$(iterator)["SugarCane"]: $(x̂i["Sugar Cane"])""")
        println( "Lower Bound: $(LB) & Uper Bound: $(UB)")
        println("===========")
    end
    @constraint(master, α >= l_i(x["Wheat"],x["Corn"],x["Sugar Cane"]))
    optimize!(master);


    iterator += 1
end

Iteration 1: x̂1["Wheat"]: 0.0 & x̂1["Corn"]: 0.0 & x̂1["SugarCane"]: 0.0
Lower Bound: -999999.0 & Uper Bound: 98000.0
Iteration 2: x̂2["Wheat"]: 88.12260536398469 & x̂2["Corn"]: 88.1226053639847 & x̂2["SugarCane"]: 88.1226053639847
Lower Bound: -132000.00000000003 & Uper Bound: -13048.531289910628
Iteration 3: x̂3["Wheat"]: 144.8562152416108 & x̂3["Corn"]: 144.8562152416108 & x̂3["SugarCane"]: 144.8562152416108
Lower Bound: -132000.00000000003 & Uper Bound: -68337.68555573832
Iteration 4: x̂4["Wheat"]: 203.96217890803112 & x̂4["Corn"]: 92.07564218393776 & x̂4["SugarCane"]: 203.96217890803115
Lower Bound: -132000.00000000003 & Uper Bound: -99788.4736026982
Iteration 5: x̂5["Wheat"]: 203.96217890803115 & x̂5["Corn"]: 8.191458857630632 & x̂5["SugarCane"]: 287.84636223433824
Lower Bound: -132000.00000000003 & Uper Bound: -99503.46602581406
Iteration 6: x̂6["Wheat"]: 169.4076573017883 & x̂6["Corn"]: 77.53884494478818 & x̂6["SugarCane"]: 253.05349775342356
Lower Bound: -118440.78431372554 &